In [1]:
from build_llm.wikipedia import baixar_wikipedia_e_salvar_com_pacote
from build_llm.pipeline import pipeline_dataset_e_embeddings

In [2]:
caminho = baixar_wikipedia_e_salvar_com_pacote(
    assunto="Machado de Assis",
    pasta_destino="../../data/dados_wiki",
    idioma="pt",
)
print(f"Arquivo salvo!")

Arquivo salvo!


In [3]:
resultado = pipeline_dataset_e_embeddings(
    caminho_texto="../../data/dados_wiki/Machado_de_Assis.txt",
    nome_base="machado_assis_wiki",

    # dataset GPT
    batch_size_gpt=8,
    max_length=128,
    stride=128,

    # treino embeddings (CBOW)
    treinar_embeddings=True,
    embedding_dim=256,
    cbow_janela=2,
    cbow_batch_size=256,
    cbow_epocas=50,
    cbow_lr=0.05,
)

print("Pasta data:", resultado["pasta_data"])
print("Vocab compacto:", resultado["vocab_size_compacto"])
print("Num batches GPT (aprox):", len(resultado["dataloader_gpt"]))

if "pasta_models" in resultado:
    print("Pasta models:", resultado["pasta_models"])
    print("Pesos token embedding:", resultado["caminho_pesos_token_embedding"])


Época 001/50 | loss médio: 8.0235
Época 010/50 | loss médio: 6.5430
Época 020/50 | loss médio: 5.8076
Época 030/50 | loss médio: 5.3651
Época 040/50 | loss médio: 5.0531
Época 050/50 | loss médio: 4.8138
Pasta data: C:\Users\Enacom\Documents\GitHub\build_llm\data\gpt_data_machado_assis_wiki
Vocab compacto: 3232
Num batches GPT (aprox): 40
Pasta models: C:\Users\Enacom\Documents\GitHub\build_llm\models\cbow_embeddings_machado_assis_wiki
Pesos token embedding: C:\Users\Enacom\Documents\GitHub\build_llm\models\cbow_embeddings_machado_assis_wiki\token_embedding_weight.pt
